In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-vbj59ycq
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/pip-req-build-vbj59ycq
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32650 sha256=37eb07a6a54f92c5970e2f333fca95b7b5fb5f2b32f4ed01cb06e61093edf7f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-wc8wejyv/wheels/d9/fb/59/7deb61b460c1c36394cd093758986ff7d36f71352dcb2e02c5
Successfully built modAL-python


In [6]:
import modAL

In [8]:
#import the packages
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from sklearn.model_selection import train_test_split
import joblib
import collections
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [9]:
mnist_data = MNIST('.', download= True, transform = ToTensor())
dataloader = DataLoader(mnist_data, shuffle=True, batch_size = 60000 )
x, y = next(iter(dataloader))

x = x.detach().cpu().numpy()
y = y.detach().cpu().numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=10000, random_state=0, stratify=y)
x_test=x_test.reshape(len(x_test), 1, 28, 28)

100%|██████████| 9912422/9912422 [00:00<00:00, 102307222.15it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 29865802.22it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 29517752.74it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9799654.72it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [10]:
#build the logistic regression model
from torch import nn
from skorch import NeuralNetClassifier
import torch
class Regression_Model(nn.Module):
    def __init__( self):
        super( Regression_Model , self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 10))

    def forward(self, x):
        out = torch.sigmoid(self.fc(x))
        return out

In [11]:
# Wrap with Skorch for compatiblity with sklearn

classifier = NeuralNetClassifier(Regression_Model,
    criterion = nn.CrossEntropyLoss,
    optimizer = torch.optim.Adam,
    train_split = None,
    verbose = 1,
    lr=0.01,
    device = device,
    )

In [12]:
#the base model
import joblib
filename = "regression_version1.joblib"
joblib.dump(classifier, filename)
#loaded_model = joblib.load(filename)

['regression_version1.joblib']

In [13]:
def CF_Print(y_test, y_pred):
    accuracy = accuracy_score(y_pred, y_test)
    CF = confusion_matrix(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    print('Confusion Matrix:')
    print(CF)
    print('Accuracy: ', accuracy)
    print('Macro F1-score: ', macro_f1)
    print('Micro F1-score: ', micro_f1)
    print('Weighted F1-score: ', weighted_f1)
    print("---------------------------------")

In [14]:
def AL_Retrain(n_initial, n_queries, instances):
    x_initial, x_pool, y_initial,  y_pool = train_test_split(x_train,y_train, train_size=n_initial, random_state=0, stratify=y_train)
    x_initial = x_initial.reshape(len(x_initial), 1, 28, 28)
    x_pool = x_pool.reshape(len(x_pool), 1, 28, 28)
    filename = "regression_version1.joblib"
    classifier = joblib.load(filename)
    learner = ActiveLearner(
        estimator = classifier,
        query_strategy = uncertainty_sampling,
        X_training = x_initial, y_training = y_initial, epochs=10,
    )
    print("No of initial data: ", n_initial)
    print(learner.score(x_test, y_test))
    y_pred = learner.predict(x_test)
    print('Confusion Matrix After trainig with initial data')
    CF_Print(y_test, y_pred)
    print("---- Train From Scratch with first model-----")
    AL_Train_Scratch(n_initial, learner)
    print("---- Teach/Retrain with new data-------")
    for idx in range(n_queries):
        print('--- Query no:  ', idx+1, ' ----')
        query_idx, query_instance = learner.query(x_pool, n_instances=instances)
        learner.teach(
            X=x_pool[query_idx],
            y=y_pool[query_idx],
        )
        print('Instance', y_pool[query_idx] )
        print(learner.score(x_test, y_test))
        x_pool = np.delete(x_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
    y_pred = learner.predict(x_test)
    print('Confusion Matrix After RE-TRAINGING')
    CF_Print(y_test, y_pred)

In [15]:
#Train the model from scratch based on the dataset selected by active learning method ensuring the uniform distribution of the data
def AL_Train_Scratch(n_initial, trained_learner):
    global x_train
    x_train_reshaped=x_train.reshape(len(x_train), 1, 28, 28)
    query_idx, query_instance = trained_learner.query(x_train_reshaped, n_instances=(n_initial*3))
    distribution=[0]*10
    flag=0
    data_per_class= int(n_initial/10)
    x_initial=[]
    y_initial=[]
    x_temp= x_train_reshaped[query_idx]
    y_temp=y_train[query_idx]
    for data, label in zip(x_temp, y_temp):
        if distribution[label]<data_per_class:
            x_initial.append(data)
            y_initial.append(label)
            distribution[label]+=1
            if distribution[label]==data_per_class:
                flag+=1
        if flag==10:
            break
    del x_temp, y_temp
    print(y_initial)
    counter = collections.Counter(y_initial)
    print(counter)
    filename = "regression_version1.joblib"
    classifier = joblib.load(filename)
    x_initial= np.array(x_initial)
    y_initial = np.array(y_initial)
    x_initial = x_initial.reshape(len(x_initial), 1, 28, 28)
    new_learner = ActiveLearner(
        estimator = classifier,
        query_strategy = uncertainty_sampling,
        X_training = x_initial, y_training = y_initial, epochs=10,
    )
    print(new_learner.score(x_test, y_test))
    y_pred = new_learner.predict(x_test)
    print("Confusion Matrix after training the model with most important dataset")
    CF_Print(y_test, y_pred)

In [16]:
AL_Retrain(30, 10, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.2880  0.1523
      2        2.1201  0.0036
      3        1.9966  0.0038
      4        1.8980  0.0049
      5        1.8150  0.0062
      6        1.7486  0.0037
      7        1.6987  0.0036
      8        1.6606  0.0034
      9        1.6294  0.0034
     10        1.6031  0.0041
No of initial data:  30
0.5967
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 713    1    4   41    1  178   37   11    1    0]
 [   0 1021    2    5    1   13    0   14   66    2]
 [  42  107  485  138    6   81   16   29   81    8]
 [  64   14   14  699    3   50    2   51  116    9]
 [  16   20   84   75  351    8  122  249   13   36]
 [  94   13    9  122    6  556    8   47   44    4]
 [  28   76    8   80    7   62  720    3    2    0]
 [  16   18    2    6    4    1   11  659    3  324]
 [  30   77   11  103   40   91    3   60  501   59]
 [  32    4   15   27  129   18   28  463   14  262]]
Accuracy:  

In [17]:
AL_Retrain(30, 10, 30)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3129  0.0025
      2        2.1399  0.0037
      3        2.0117  0.0037
      4        1.9119  0.0043
      5        1.8273  0.0039
      6        1.7581  0.0035
      7        1.7055  0.0036
      8        1.6662  0.0037
      9        1.6347  0.0042
     10        1.6081  0.0030
No of initial data:  30
0.5942
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 692    1    4   41    0  206   33    6    4    0]
 [   0 1019    2    2    1   16    0   18   65    1]
 [  35  129  478  122   10   78   22   28   82    9]
 [  39   11   13  693    4   56    2   63  131   10]
 [  13   12   91   78  363   12  101  264    7   33]
 [  56   17    8  116   12  573    7   44   66    4]
 [  21   84   16   73    8   80  700    2    2    0]
 [  19   21    3    7    7    1    8  665    3  310]
 [  17   85   12   85   32  100    3   64  517   60]
 [  32    6   14   29  153   21   24  457   14  242]]
Accuracy:  

In [18]:
AL_Retrain(50, 10, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3007  0.0026
      2        2.1504  0.0053
      3        2.0347  0.0035
      4        1.9414  0.0047
      5        1.8616  0.0036
      6        1.7953  0.0046
      7        1.7438  0.0052
      8        1.7044  0.0039
      9        1.6723  0.0044
     10        1.6446  0.0051
No of initial data:  50
0.6649
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 871    0    9   37    0   28   21    0   12    9]
 [   0 1060    4   15   10    3    0    8   24    0]
 [  68   45  579  171   15   21    7   27   57    3]
 [  18   16   15  821    6   29    2   39   69    7]
 [  46   18   45    8  631    1   41  111    1   72]
 [  52   29   13  183   36  406    5   30  138   11]
 [  40   82   30   48    7   57  684    3   30    5]
 [  28   23    5    6   23    0    3  751    4  201]
 [  34   89   35   92   49   54    2   31  538   51]
 [  36    3   23   13  388    1    6  209    5  308]]
Accuracy:  

In [19]:
AL_Retrain(50, 10, 30)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3134  0.0043
      2        2.1617  0.0051
      3        2.0448  0.0042
      4        1.9500  0.0045
      5        1.8688  0.0062
      6        1.8012  0.0036
      7        1.7481  0.0038
      8        1.7076  0.0043
      9        1.6756  0.0033
     10        1.6482  0.0033
No of initial data:  50
0.6713
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 871    0   14   22    1   29   28    0   13    9]
 [   0 1060    5   14    7    2    0   15   21    0]
 [  64   44  634  108   15   23   15   23   63    4]
 [  22   22   19  809    7   34    0   33   66   10]
 [  30   14   52    7  647    1   50  103    1   69]
 [  58   26   18  170   43  385    5   30  148   20]
 [  33   79   36   42    7   41  706    3   34    5]
 [  25   24   11    4   23    1    5  751    2  198]
 [  28   98   47   66   52   56    1   28  551   48]
 [  33    2   27   10  407    0    7  201    6  299]]
Accuracy:  

In [20]:
AL_Retrain(100, 10, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3030  0.0043
      2        2.1643  0.0063
      3        2.0559  0.0051
      4        1.9677  0.0052
      5        1.8934  0.0049
      6        1.8333  0.0054
      7        1.7868  0.0045
      8        1.7504  0.0049
      9        1.7203  0.0052
     10        1.6945  0.0044
No of initial data:  100
0.727
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 796    0   12   22    3   57   73    0   16    8]
 [   0 1029   11    4    1    1    0    1   75    2]
 [  14   49  765   41   24    6   19   15   54    6]
 [  14   28   35  815    3   34   11   27   44   11]
 [   7   15   21    0  765    2   22   17    1  124]
 [  17   14   21  267   39  459   16    3   39   28]
 [  21   25   40    3   10   12  873    0    2    0]
 [  16   16    5    5   58    4    3  688   16  233]
 [   7   72   27   81   39   69    5    5  601   69]
 [  15   10   11   13  371    3    3   75   12  479]]
Accuracy:  

In [21]:
AL_Retrain(100, 10, 30)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.2926  0.0030
      2        2.1579  0.0050
      3        2.0525  0.0070
      4        1.9645  0.0042
      5        1.8902  0.0044
      6        1.8311  0.0060
      7        1.7859  0.0095
      8        1.7500  0.0044
      9        1.7199  0.0053
     10        1.6938  0.0044
No of initial data:  100
0.7297
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 802    0    9   18    3   55   67    0   24    9]
 [   0 1029    6    4    0    1    0    2   81    1]
 [  18   47  753   43   26    8   17   16   59    6]
 [  13   28   32  818    3   31   12   21   52   12]
 [   8   14   16    1  773    3   22   18    3  116]
 [  18   15   21  239   41  467   15    3   59   25]
 [  25   20   45    3    7   16  868    0    2    0]
 [  17   15    5    6   53    5    3  672   16  252]
 [   8   61   25   63   36   70    5    7  640   60]
 [  15    9    9   13  375    6    2   76   12  475]]
Accuracy: 

In [22]:
AL_Retrain(300, 10, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.2390  0.0100
      2        1.9814  0.0152
      3        1.8385  0.0128
      4        1.7622  0.0125
      5        1.7131  0.0116
      6        1.6802  0.0123
      7        1.6562  0.0112
      8        1.6372  0.0116
      9        1.6224  0.0153
     10        1.6101  0.0161
No of initial data:  300
0.7964
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 904    1    8   28    2    3   31    0   10    0]
 [   0 1090    4    7    0   17    1    1    3    1]
 [  20   16  803   16   23    5   24   42   42    2]
 [  21   14   24  804    1   59    3   40   44   12]
 [   7   16    7    1  718    5   31    6   13  170]
 [  41   50    9  120   14  588   25    2   43   11]
 [  30   22   40    4   11   13  860    4    2    0]
 [  15   26    6    1   16    2    1  844   15  118]
 [  18  126   29   42    7   61    4    5  660   23]
 [  19   19    8   11  143   11    5   59   24  693]]
Accuracy: 

In [23]:
AL_Retrain(300, 10, 30)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.2389  0.0106
      2        1.9792  0.0121
      3        1.8378  0.0102
      4        1.7611  0.0104
      5        1.7123  0.0098
      6        1.6796  0.0093
      7        1.6557  0.0108
      8        1.6367  0.0104
      9        1.6217  0.0099
     10        1.6094  0.0104
No of initial data:  300
0.7956
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 902    1    9   28    2    3   32    0   10    0]
 [   0 1092    4    5    0   17    1    1    3    1]
 [  20   18  801   16   25    4   21   40   46    2]
 [  25   13   28  798    1   57    3   42   42   13]
 [   7   12    7    1  705    4   30    7   14  187]
 [  37   44   10  115   14  601   27    2   44    9]
 [  33   19   40    7   11   13  860    1    2    0]
 [  15   25    7    2   16    2    1  837   17  122]
 [  17  119   33   39    7   62    4    6  666   22]
 [  18   16    8   10  147    9    4   63   23  694]]
Accuracy: 